<a href="https://colab.research.google.com/github/yangju0411/KaggleKr_4th/blob/main/kakr4_lgbm_rf_xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/kakr-4th-competition/sample_submission.csv
/kaggle/input/kakr-4th-competition/test.csv
/kaggle/input/kakr-4th-competition/train.csv


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.model_selection import KFold
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import VotingClassifier
import pickle
import joblib
import gc

In [ ]:
train = pd.read_csv('/kaggle/input/kakr-4th-competition/train.csv', index_col = "id")
test = pd.read_csv('/kaggle/input/kakr-4th-competition/test.csv', index_col = "id")

id를 행이름으로 사용

In [ ]:
y = train['income'] == '>50K'
X = train.drop(['income'], axis=1)

prediction : [소득이 50,000$을 넘는다면 1, 아니라면 0] <br>
타겟을 T&F 형태로

## OneHot Encoding

In [ ]:
'''import category_encoders as ce

ohe = ce.OneHotEncoder(use_cat_names = True)
ohe.fit(X)
X = ohe.transform(X)
test_X = ohe.transform(test)'''

'import category_encoders as ce\n\nohe = ce.OneHotEncoder(use_cat_names = True)\nohe.fit(X)\nX = ohe.transform(X)\ntest_X = ohe.transform(test)'

원핫인코딩 시 스코어가 더 떨어져서 라벨로 다시 돌아왔습니다.

## Label Encoding

In [ ]:
from category_encoders.ordinal import OrdinalEncoder
le = OrdinalEncoder(list(X.columns))

X = le.fit_transform(X, y)
test_X = le.transform(test)

In [ ]:
X.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country
id,,,,,,,,,,,,,,
0,40,1,168538,1,9,1,1,1,1,1,0,0,60,1
1,17,1,101626,2,5,2,2,2,1,1,0,0,20,1
2,18,1,353358,3,10,2,3,2,1,1,0,0,16,1
3,21,1,151158,3,10,2,4,2,1,2,0,0,25,1
4,24,1,122234,3,10,2,5,3,2,2,0,0,20,2


In [ ]:
X.shape

(26049, 14)

In [ ]:
test_X.shape

(6512, 14)

# LGBM

In [ ]:
NFOLDS = 5
folds = KFold(n_splits = NFOLDS)

columns = X.columns
splits = folds.split(X, y)
lgbm_preds = np.zeros(test_X.shape[0])

In [ ]:
lgbm = LGBMClassifier(objective='binary', verbose = 1, random_state = 17,
                     boosting_type = 'gbdt', learning_rate = 0.1)


for fold_n, (train_index, valid_index) in enumerate(splits):
    print('Fold: ', fold_n+1)
    X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]

    evals = [(X_train, y_train), (X_valid, y_valid)]
    lgbm.fit(X_train, y_train, eval_metric='f1', eval_set=evals, verbose=True)
        
    lgbm_preds += lgbm.predict(test_X).astype(int) / NFOLDS
    
    del X_train, X_valid, y_train, y_valid
    gc.collect()

Fold:  1
[1]	training's binary_logloss: 0.51029	valid_1's binary_logloss: 0.507521
[2]	training's binary_logloss: 0.477769	valid_1's binary_logloss: 0.475498
[3]	training's binary_logloss: 0.451924	valid_1's binary_logloss: 0.450099
[4]	training's binary_logloss: 0.431056	valid_1's binary_logloss: 0.429931
[5]	training's binary_logloss: 0.413344	valid_1's binary_logloss: 0.412961
[6]	training's binary_logloss: 0.39845	valid_1's binary_logloss: 0.398591
[7]	training's binary_logloss: 0.385568	valid_1's binary_logloss: 0.385989
[8]	training's binary_logloss: 0.374398	valid_1's binary_logloss: 0.375523
[9]	training's binary_logloss: 0.36472	valid_1's binary_logloss: 0.36682
[10]	training's binary_logloss: 0.35611	valid_1's binary_logloss: 0.358814
[11]	training's binary_logloss: 0.348373	valid_1's binary_logloss: 0.351625
[12]	training's binary_logloss: 0.341334	valid_1's binary_logloss: 0.345092
[13]	training's binary_logloss: 0.335245	valid_1's binary_logloss: 0.339384
[14]	training's b

https://www.kaggle.com/werooring/basic-eda-lgbm-modeling-public-score-0-87714

In [ ]:
def make_sub(predicts, filename):
    sample_submission = pd.read_csv('/kaggle/input/kakr-4th-competition/sample_submission.csv')
    sample_submission['prediction'] = predicts

    for ix, row in sample_submission.iterrows():
        if row['prediction'] > 0.5:
            sample_submission.loc[ix, 'prediction'] = 1
        else:
            sample_submission.loc[ix, 'prediction'] = 0

    sample_submission = sample_submission.astype({"prediction": int})
    sample_submission.to_csv(path_or_buf = filename, index=False)

확률에서 예측 결과를 만들어주는 함수

In [ ]:
make_sub(lgbm_preds, "LGBMsubmissions.csv")

# XGB

In [ ]:
XGB = XGBClassifier(gamma = 0.01, learning_rate = 0.1,
                    max_depth = 7, min_child_weight = 3, random_state = 17)

gridsearch로 찾은 하이퍼 파라미터 넣어줌

In [ ]:
NFOLDS = 5
folds = KFold(n_splits = NFOLDS)

columns = X.columns
splits = folds.split(X, y)
XGB_preds = np.zeros(test_X.shape[0])

In [ ]:
def f1_eval(y_pred, dtrain):
    y_true = dtrain.get_label()
    err = 1-f1_score(y_true, np.round(y_pred), average = "micro")
    return 'f1_err', err

In [ ]:
for fold_n, (train_index, valid_index) in enumerate(splits):
    print('Fold: ', fold_n+1)
    X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]

    evals = [(X_train, y_train), (X_valid, y_valid)]
    XGB.fit(X_train, y_train, eval_metric=f1_eval, eval_set=evals, verbose=True)
        
    XGB_preds += XGB.predict_proba(test_X)[:, 1] / NFOLDS
    
    del X_train, X_valid, y_train, y_valid
    gc.collect()

Fold:  1
[0]	validation_0-error:0.14180	validation_1-error:0.15509	validation_0-f1_err:0.24267	validation_1-f1_err:0.23954
[1]	validation_0-error:0.13883	validation_1-error:0.15086	validation_0-f1_err:0.24267	validation_1-f1_err:0.23954
[2]	validation_0-error:0.13854	validation_1-error:0.14357	validation_0-f1_err:0.58482	validation_1-f1_err:0.58215
[3]	validation_0-error:0.13979	validation_1-error:0.14741	validation_0-f1_err:0.70090	validation_1-f1_err:0.69482
[4]	validation_0-error:0.13720	validation_1-error:0.14626	validation_0-f1_err:0.72532	validation_1-f1_err:0.71689
[5]	validation_0-error:0.13950	validation_1-error:0.14683	validation_0-f1_err:0.74749	validation_1-f1_err:0.73935
[6]	validation_0-error:0.13950	validation_1-error:0.14645	validation_0-f1_err:0.77580	validation_1-f1_err:0.77044
[7]	validation_0-error:0.13767	validation_1-error:0.14530	validation_0-f1_err:0.77753	validation_1-f1_err:0.77121
[8]	validation_0-error:0.13729	validation_1-error:0.14395	validation_0-f1_err:0

In [ ]:
make_sub(predicts = XGB_preds, filename = "XGBsubmissions.csv")

# RF

In [ ]:
rf = RandomForestClassifier(max_features = 3, min_samples_split = 8,
                           n_estimators = 100, criterion = "entropy",
                           bootstrap = True)

gridsearch로 찾아낸 하이퍼파라미터

In [ ]:
rf.fit(X, y)

RandomForestClassifier(criterion='entropy', max_features=3, min_samples_split=8)

In [ ]:
RF_preds = rf.predict_proba(test_X)[:, 1]

In [ ]:
make_sub(RF_preds, "rf.csv")

# Ensemble

## 산술평균

In [ ]:
ensemble_arith_1 = (XGB_preds + lgbm_preds + RF_preds) / 3

In [ ]:
make_sub(predicts = ensemble_arith_1, filename = "Ensemble_arith_1.csv")

In [ ]:
ensemble_arith_2 = (XGB_preds + lgbm_preds) / 2

In [ ]:
make_sub(predicts = ensemble_arith_2, filename = "Ensemble_arith_2.csv")

## 기하평균

In [ ]:
ensemble_geo_1 = (XGB_preds * lgbm_preds * RF_preds) ** (1/ 3)

In [ ]:
ensemble_geo_2 = (XGB_preds * lgbm_preds) ** (1/ 2)

기하평균을 비율에 쓴다고 배워서 확률에도 알맞지 않을까 싶어서 한 번 해봤습니다.

In [ ]:
make_sub(predicts = ensemble_geo_1, filename = "Ensemble_geo_1.csv")
make_sub(predicts = ensemble_geo_2, filename = "Ensemble_geo_2.csv")

## voting classifier

### Hard Voting

In [ ]:
voting_hard = VotingClassifier(estimators = [("lgbm", lgbm), ("xgb", XGB), ("rf", rf)],
                         voting = "hard")

In [ ]:
voting_hard = voting_hard.fit(X, y)

In [ ]:
hard_preds = voting_hard.predict(test_X)

In [ ]:
make_sub(hard_preds, "voting_hard_1.csv")

In [ ]:
voting_hard = VotingClassifier(estimators = [("lgbm", lgbm), ("xgb", XGB)],
                         voting = "hard")
voting_hard = voting_hard.fit(X, y)
hard_preds = voting_hard.predict(test_X)
make_sub(hard_preds, "voting_hard_2.csv")

### Soft

In [ ]:
voting_soft = VotingClassifier(estimators = [("lgbm", lgbm), ("xgb", XGB), ("rf", rf)],
                         voting = "soft")

In [ ]:
voting_soft = voting_soft.fit(X, y)

In [ ]:
soft_preds = voting_soft.predict(test_X)

In [ ]:
make_sub(soft_preds, "voting_soft_1.csv")

In [ ]:
voting_soft = VotingClassifier(estimators = [("lgbm", lgbm), ("xgb", XGB), ("rf", rf)],
                         voting = "soft")
voting_soft = voting_soft.fit(X, y)
soft_preds = voting_soft.predict(test_X)
make_sub(soft_preds, "voting_soft_2.csv")

voting은 fold하지 않아서 평균을 내는 것보다 오히려 더 떨어진 것 같다.<br>
기하평균이 더 민감한 듯 하다.<br>
예측력이 낮은 랜덤포레스트는 제거하는 쪽이 더 나은 것 같다.<br>
다음에는 catboost를 한 번 시도해보자

**참고**<br>
https://www.kaggle.com/subinium/kakr-eda
https://www.kaggle.com/werooring/basic-eda-lgbm-modeling-public-score-0-87714
https://www.kaggle.com/werooring/4th-place-solution-simple-lgbm
https://www.kaggle.com/c/kakr-4th-competition/discussion/196061